In [12]:
sys.path.append("../pytorch3dunet/unet3d")
sys.path.append("../src/utils/")


import pydicom
import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F

from tqdm import tqdm
from model import ResidualUNet3D
from metrics import DiceCoefficient, MeanIoU
from my_losses import MyDiceLoss, CategoricalCrossEntropyLoss
from base_scripts import create_fold_from_data, SegmDataset

import ipyvolume as ipv
import nibabel as nib
import torch


import os
import random
import sys

In [2]:
from losses import DiceLoss

In [3]:
df = create_fold_from_data(
    path_data="../src/data_for_segmentation.csv",
)

In [4]:
dataset = SegmDataset(
    data=df,
    is_train=True,
    size=(128, 128, 64),
    path_to_folder_with_imgs=r"C:\KAGGLE\MEDICINE\data\train_images",
    path_to_folder_with_masks=r"C:\KAGGLE\MEDICINE\data\segmentations",
    typ_augm="v1",
)

# My custom loss 

In [5]:
model = ResidualUNet3D(
    in_channels=1, out_channels=5, f_maps=[16, 32, 64, 128, 256], final_sigmoid=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
loss_fn = MyDiceLoss()

In [7]:
list_y_true = None
list_y_pred = None
print("loss", loss_fn)
for i in range(5):
    model.train()
    ind = 1
    img, mask = dataset[ind]
    tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
    tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
    optimizer.zero_grad()
    pred = model.forward(tensor_img)

    # predicted_probs = pred

    # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

    loss = loss_fn(pred, tensor_mask.long())

    print("TRAIN", ind, "loss", loss)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        ind = 10
        img, mask = dataset[ind]
        tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
        tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
        # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

        pred = model.forward(tensor_img)

        # predicted_probs = pred
        loss = loss_fn(pred, tensor_mask.long())

        y_preds = pred.to("cpu").numpy()

        target_one_hot = tensor_mask
        mask = target_one_hot.to("cpu").numpy()
        if list_y_pred is None:
            list_y_pred = y_preds
            list_y_true = mask

        else:
            list_y_pred = np.vstack((list_y_pred, y_preds))
            list_y_true = np.vstack((list_y_true, mask))
        print("TEST", ind, "loss", i, loss)

loss MyDiceLoss()
TRAIN 1 loss tensor(0.8803, device='cuda:0', grad_fn=<RsubBackward1>)
TEST 10 loss 0 tensor(0.9076, device='cuda:0')
TRAIN 1 loss tensor(0.8624, device='cuda:0', grad_fn=<RsubBackward1>)
TEST 10 loss 1 tensor(0.9040, device='cuda:0')
TRAIN 1 loss tensor(0.8449, device='cuda:0', grad_fn=<RsubBackward1>)
TEST 10 loss 2 tensor(0.9006, device='cuda:0')
TRAIN 1 loss tensor(0.8293, device='cuda:0', grad_fn=<RsubBackward1>)
TEST 10 loss 3 tensor(0.8971, device='cuda:0')
TRAIN 1 loss tensor(0.8137, device='cuda:0', grad_fn=<RsubBackward1>)
TEST 10 loss 4 tensor(0.8936, device='cuda:0')


 # CategoricalCrossEntropyLoss

In [8]:
model = ResidualUNet3D(
    in_channels=1, out_channels=5, f_maps=[16, 32, 64, 128, 256], final_sigmoid=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
loss_fn = CategoricalCrossEntropyLoss()

list_y_true = None
list_y_pred = None
print("loss", loss_fn)
for i in range(10):
    model.train()
    ind = 1
    img, mask = dataset[ind]
    tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
    tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
    optimizer.zero_grad()
    pred = model.forward(tensor_img)

    # predicted_probs = pred

    # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

    loss = loss_fn(pred, tensor_mask.long())

    print("TRAIN", ind, "loss", loss)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        ind = 10
        img, mask = dataset[ind]
        tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
        tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
        # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

        pred = model.forward(tensor_img)

        # predicted_probs = pred
        loss = loss_fn(pred, tensor_mask.long())

        y_preds = pred.to("cpu").numpy()

        target_one_hot = tensor_mask
        mask = target_one_hot.to("cpu").numpy()
        if list_y_pred is None:
            list_y_pred = y_preds
            list_y_true = mask

        else:
            list_y_pred = np.vstack((list_y_pred, y_preds))
            list_y_true = np.vstack((list_y_true, mask))
        print("TEST", ind, "loss", i, loss)

loss CategoricalCrossEntropyLoss()
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 0 tensor(1.4726, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 1 tensor(1.4366, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 2 tensor(1.3801, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 3 tensor(1.3454, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 4 tensor(1.4178, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 5 tensor(1.4303, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 6 tensor(1.4297, device='cuda:0')
TRAIN 1 loss tensor(nan, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 7 tensor(1.4139, device='cuda:0')
TRAIN 1 loss 

 # DiceLoss

In [1]:
model = ResidualUNet3D(
    in_channels=1, out_channels=5, f_maps=[16, 32, 64, 128, 256], final_sigmoid=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
loss_fn = DiceLoss(normalization="softmax", num_classes=5)

list_y_true = None
list_y_pred = None
print("loss", loss_fn)
for i in range(10):
    model.train()
    ind = 1
    img, mask = dataset[ind]
    tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
    tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
    optimizer.zero_grad()
    pred = model.forward(tensor_img)

    # predicted_probs = pred

    # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

    loss = loss_fn(pred, tensor_mask.long())

    print("TRAIN", ind, "loss", loss)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        ind = 10
        img, mask = dataset[ind]
        tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
        tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
        # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

        pred = model.forward(tensor_img)

        # predicted_probs = pred
        loss = loss_fn(pred, tensor_mask.long())

        y_preds = pred.to("cpu").numpy()

        target_one_hot = tensor_mask
        mask = target_one_hot.to("cpu").numpy()
        if list_y_pred is None:
            list_y_pred = y_preds
            list_y_true = mask

        else:
            list_y_pred = np.vstack((list_y_pred, y_preds))
            list_y_true = np.vstack((list_y_true, mask))
        print("TEST", ind, "loss", i, loss)

NameError: name 'ResidualUNet3D' is not defined

nn.CrossEntropyLoss()

In [10]:
model = ResidualUNet3D(
    in_channels=1, out_channels=5, f_maps=[16, 32, 64, 128, 256], final_sigmoid=False
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
loss_fn = torch.nn.CrossEntropyLoss()

list_y_true = None
list_y_pred = None
print("loss", loss_fn)
for i in range(10):
    model.train()
    ind = 1
    img, mask = dataset[ind]
    tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
    tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
    optimizer.zero_grad()
    pred = model.forward(tensor_img)

    # predicted_probs = pred

    # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

    loss = loss_fn(pred, tensor_mask.long())

    print("TRAIN", ind, "loss", loss)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        ind = 10
        img, mask = dataset[ind]
        tensor_mask = torch.Tensor(mask).unsqueeze(0).to(device)
        tensor_img = torch.Tensor(img).unsqueeze(0).unsqueeze(0).to(device)
        # tensor_mask = F.one_hot(tensor_mask.long(), num_classes=predicted_probs.long().shape[1]).permute(0, 4, 1, 2, 3).float()

        pred = model.forward(tensor_img)

        # predicted_probs = pred
        loss = loss_fn(pred, tensor_mask.long())

        y_preds = pred.to("cpu").numpy()

        target_one_hot = tensor_mask
        mask = target_one_hot.to("cpu").numpy()
        if list_y_pred is None:
            list_y_pred = y_preds
            list_y_true = mask

        else:
            list_y_pred = np.vstack((list_y_pred, y_preds))
            list_y_true = np.vstack((list_y_true, mask))
        print("TEST", ind, "loss", i, loss)

loss CrossEntropyLoss()
TRAIN 1 loss tensor(1.3133, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 0 tensor(1.4329, device='cuda:0')
TRAIN 1 loss tensor(1.0200, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 1 tensor(1.3112, device='cuda:0')
TRAIN 1 loss tensor(0.7612, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 2 tensor(1.1936, device='cuda:0')
TRAIN 1 loss tensor(0.5745, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 3 tensor(1.1110, device='cuda:0')
TRAIN 1 loss tensor(0.4728, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 4 tensor(1.0585, device='cuda:0')
TRAIN 1 loss tensor(0.4234, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 5 tensor(1.0278, device='cuda:0')
TRAIN 1 loss tensor(0.3988, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 6 tensor(1.0116, device='cuda:0')
TRAIN 1 loss tensor(0.3803, device='cuda:0', grad_fn=<NllLoss2DBackward0>)
TEST 10 loss 7 tensor(1.0065, device='cuda:0')


# Базовая метрика из либы Unet3D

In [11]:
torch.Tensor(list_y_pred).shape, F.one_hot(
    torch.Tensor(list_y_true).long(), num_classes=5
).permute(0, 4, 1, 2, 3).float().shape

(torch.Size([10, 5, 128, 128, 64]), torch.Size([10, 5, 128, 128, 64]))

In [49]:
metric = MeanIoU(skip_channels=([0, 3]))
metric(
    torch.Tensor(list_y_pred),
    F.one_hot(torch.Tensor(list_y_true).long(), num_classes=5)
    .permute(0, 4, 1, 2, 3)
    .float(),
)

tensor(0.0020)

In [17]:
metric = DiceCoefficient()
metric(
    torch.Tensor(list_y_pred),
    F.one_hot(torch.Tensor(list_y_true).long(), num_classes=5)
    .permute(0, 4, 1, 2, 3)
    .float(),
)

tensor(0.2470)

In [58]:
def print_metric_MeanIoU(y_pred, y_true):
    metric = MeanIoU(skip_channels=())
    y_pred = torch.Tensor(list_y_pred)
    y_true = (
        F.one_hot(torch.Tensor(list_y_true).long(), num_classes=5)
        .permute(0, 4, 1, 2, 3)
        .float()
    )
    print(f"MeanIoU : {metric(y_pred, y_true)}")

    metric = MeanIoU(skip_channels=[0])
    print(f"MeanIoU without 0 channel: {metric(y_pred, y_true):.3f}")
    metric = MeanIoU(skip_channels=((0, 5)))
    print(f"MeanIoU without 0, 5 channel: {metric(y_pred, y_true):.3f}")

In [59]:
print_metric_MeanIoU(list_y_pred, list_y_true)

MeanIoU : 0.18886908888816833
MeanIoU without 0 channel: 0.002
MeanIoU without 0, 5 channel: 0.002
